In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
print("TensorFlow version:", tf.__version__)
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
import shutil

TensorFlow version: 2.18.0


In [2]:
 %reload_ext tensorboard
from tensorflow.keras import Model
from pathlib import Path
import pandas as pd

In [3]:
BATCH_SIZE = 64
BUFFER_SIZE = 128
LEARNING_RATE = 0.05
EPOCHS = 50

In [5]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [6]:
X_train.shape

(436, 8)

In [7]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.int64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

In [73]:
@tf.keras.utils.register_keras_serializable()
class SomeModel(Model):
    def __init__(self, neurons_cnt=128, **kwargs):
        super(SomeModel, self).__init__(**kwargs)
        self.neurons_cnt = neurons_cnt  # Сохраняем значение параметра для конфигурации
        self.d_in = Dense(8, activation='relu')
        self.d1 = Dense(neurons_cnt, activation='relu')
        self.d2 = Dense(neurons_cnt, activation='relu')
        self.d_out = Dense(1)

    def call(self, x):
        x = self.d_in(x)
        x = self.d1(x)
        x = self.d2(x)
        return self.d_out(x)

    def get_config(self):
        # Возвращаем параметры модели, включая кастомные
        config = super(SomeModel, self).get_config()
        config.update({
            "neurons_cnt": self.neurons_cnt  # Добавляем кастомный параметр в конфигурацию
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Создаём экземпляр класса из конфигурации
        return cls(**config)

In [75]:
# Create an instance of the model
model = SomeModel(neurons_cnt=64)
model.build(input_shape=(None, 8))

C:\Users\ru-lo\PycharmProjects\tpu-8e21-ai-basis\venv\Lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'some_model_6', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [76]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)

In [77]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_mae')

In [78]:
@tf.function
def train_step(input_vector, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(input_vector, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [79]:
@tf.function
def test_step(input_vector, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(input_vector, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [80]:
from tensorflow import keras

In [90]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = logs_path / 'gradient_tape' / current_time / 'train'
train_log_dir.mkdir(exist_ok=True, parents=True)
test_log_dir = logs_path / 'gradient_tape' / current_time / 'test'
test_log_dir.mkdir(exist_ok=True, parents=True)
train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))

logdir=logs_path / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(exist_ok=True, parents=True)
fit_summary_writer = tf.summary.create_file_writer(str(logdir))

tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=str(logdir))
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  for (x_train, y_train) in train_ds:

    with fit_summary_writer.as_default():
      train_step(x_train, y_train)


  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_ds:
    test_step(x_test, y_test)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('mae', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test MAE: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result(),
                         test_loss.result(),
                         test_accuracy.result()))

  # Reset metrics every epoch
  train_loss.reset_state()
  test_loss.reset_state()
  train_accuracy.reset_state()
  test_accuracy.reset_state()

with fit_summary_writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=str(logdir)
  )

Epoch 1, Loss: 0.6362075805664062, Accuracy: 0.603752613067627, Test Loss: 0.6878835558891296, Test MAE: 0.5995654463768005
Epoch 2, Loss: 0.5787534117698669, Accuracy: 0.5696507096290588, Test Loss: 0.7337146401405334, Test MAE: 0.5701504349708557
Epoch 3, Loss: 0.5713530778884888, Accuracy: 0.5627707242965698, Test Loss: 0.6975072622299194, Test MAE: 0.5809425115585327
Epoch 4, Loss: 0.5715385675430298, Accuracy: 0.5714064836502075, Test Loss: 0.9550483226776123, Test MAE: 0.6408940553665161
Epoch 5, Loss: 0.6469525098800659, Accuracy: 0.5931909680366516, Test Loss: 0.6891505718231201, Test MAE: 0.5929657816886902
Epoch 6, Loss: 0.6046574711799622, Accuracy: 0.5864913463592529, Test Loss: 0.6879726648330688, Test MAE: 0.6031933426856995
Epoch 7, Loss: 0.5803183913230896, Accuracy: 0.5739027261734009, Test Loss: 0.7313417792320251, Test MAE: 0.5702424645423889
Epoch 8, Loss: 0.5844343304634094, Accuracy: 0.5744612216949463, Test Loss: 0.7184798121452332, Test MAE: 0.5716366767883301
E

UnavailableError: Cannot export profiling results. No profiler is running.

In [91]:
model.save('./data/models/mymodel.keras')

In [92]:
loaded_model = keras.models.load_model('./data/models/mymodel.keras')

In [93]:
loaded_model

<SomeModel name=some_model_6, built=True>

In [94]:
model

<SomeModel name=some_model_6, built=True>

In [95]:
# Let's check:
import numpy as np
np.testing.assert_allclose(
    model.predict(X_test),
    loaded_model.predict(X_test)
)
%tensorboard --logdir ./data/logs

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


Reusing TensorBoard on port 6006 (pid 23644), started 0:09:25 ago. (Use '!kill 23644' to kill it.)

In [19]:
%tensorboard --logdir ./data/logs/gradient_tape

Launching TensorBoard...